* produce indivual gpx files (XML) aligned with 3d routes (json)

In [8]:
import os
import pandas as pd

In [9]:
import json
import gpxpy.gpx
from skitracker_utils import normalise_run, prepare_data, find_routes

In [10]:
os.listdir('raw_split_files//')

['sam_FR_10Apr20.pkl',
 'Josh_FR_26Feb23.pkl',
 'Josh_FR_27Feb23.pkl',
 'Josh_FR_28Feb23.pkl',
 'josh_BG_17Jan20.pkl',
 'sam_FR_09Apr20.pkl',
 'sam_FR_08Apr20.pkl',
 'jon_BG_18Jan20.pkl']

In [11]:
raw_pickle = 'Josh_FR_26Feb23.pkl'
output_dir = 'split_screen_files/'
input_dir = 'raw_split_files/'

In [12]:
raw = pd.read_pickle(input_dir + raw_pickle)

In [13]:
df = prepare_data(raw)
runs = find_routes(df)


In [15]:
for i, run in enumerate(runs):
    prefix = f'test_{i}'
    
    save_gpx_and_json(run, prefix, output_dir)

In [14]:
def save_gpx_and_json(run, prefix, output_dir):
    
    run_data = run['run_data']
    
    # gpx / map file
    gpx = gpxpy.gpx.GPX()
    # Create first track in our GPX:
    gpx_track = gpxpy.gpx.GPXTrack()
    gpx.tracks.append(gpx_track)
    
    gpx_segment = gpxpy.gpx.GPXTrackSegment()
    gpx_track.segments.append(gpx_segment)


    points = run_data[['Lat', 'Lon', 'GPS_Alt']].values
    # create points
    for point in points:
        gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(point[0], point[1], elevation=point[2]))

    xml = gpx.to_xml()
    
    out_filename_gpx = prefix + '_gpx.xml'
    with open(output_dir + out_filename_gpx, 'w') as f:
        f.write(xml)
    
    # json / 3d file
    run_data.rename(columns={'Lat': 'x', 'Lon': 'y', 'GPS_Alt': 'z', 'grad_colour': 'c'}, inplace=True)

    colours = run_data['c'].copy()

    values = normalise_run(run_data[['x', 'y', 'z']])

    out_data = pd.concat((values, colours), axis=1)

    json_values = out_data.to_dict(orient='records')

    json_data = {}
    json_data['scale'] = run['scale']
    json_data['values'] = json_values

    out_filename_json = prefix + '.json'
    with open(output_dir + out_filename_json, 'w') as f:
        json.dump(json_data, f)
    